In [1]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.utils import resample

In [3]:
train_df = pd.read_csv('/Users/tuxiaochi/Desktop/jupyternb/we_data/train.csv')
train_df.head()

,click,weekday,hour,bidid,userid,useragent,IP,region,city,adexchange,...,slotheight,slotvisibility,slotformat,slotprice,creative,bidprice,payprice,keypage,advertiser,usertag
0,0,5,22,b7bea80521fdecd95d2d761a38c91c3f09618066,2e880fb7d690cf7377b2e42e701728e3f3c0e4c1,windows_ie,125.37.175.*,2,2,2.0,...,200,2,0,5,a4f763f78ef3eedfe614263b94a8924e,238,5,0f951a030abdaedd733ee8d114ce2944,3427,NaN
1,0,1,20,4f51205475678f5a124bc76b2c54163bf8eaa7eb,3a1fe01360ff8100e7d006b83b77a3e4c01d928c,windows_chrome,171.36.92.*,238,239,1.0,...,250,FourthView,Na,0,10722,294,23,NaN,2821,NaN
2,0,3,13,b604e3fd054a658ab7ced4285ebf2ef54d2bd890,801d18a056b6fe6b06a794aef17fb0d6daff2414,windows_ie,59.46.106.*,40,41,2.0,...,250,2,0,5,798b2d49952d77f1eace9f23c210d0b5,238,24,0f951a030abdaedd733ee8d114ce2944,3427,"10052,10006,13866,10110"
3,0,6,23,0348beeae93e561584c3b50fc9e7746a33048ad7,0d6eaf2259699990e38a1fc5116f112070b9ecdc,windows_ie,114.250.226.*,1,1,1.0,...,600,2,1,0,cb7c76e7784031272e37af8e7e9b062c,300,25,bebefa5efe83beee17a3d245e7c5085b,1458,"13866,10063,10111"
4,0,5,6,268149c1789bce2bc9798ffd97ec431219bafeb3,a239d9bb642460d974ba67f85e63b8d3e214da0e,windows_ie,183.63.192.*,216,233,2.0,...,90,OtherView,Na,133,7330,277,133,NaN,2259,NaN


In [4]:
cat_vars=['region','weekday','useragent','adexchange','slotwidth','slotheight','slotvisibility','advertiser']

for i in cat_vars:
    one_hot = pd.get_dummies(train_df[i],prefix=i)
    train_df = train_df.drop(i,axis = 1)
    train_df = train_df.join(one_hot)

train_df.head()

,click,hour,bidid,userid,IP,city,domain,url,urlid,slotid,...,slotvisibility_ThirdView,advertiser_1458,advertiser_2259,advertiser_2261,advertiser_2821,advertiser_2997,advertiser_3358,advertiser_3386,advertiser_3427,advertiser_3476
0,0,22,b7bea80521fdecd95d2d761a38c91c3f09618066,2e880fb7d690cf7377b2e42e701728e3f3c0e4c1,125.37.175.*,2,trqRTvKaXTKfgg24JKTI,6447a7dfa30fe6eb410c91860e7c9b45,NaN,2015392487,...,0,0,0,0,0,0,0,0,1,0
1,0,20,4f51205475678f5a124bc76b2c54163bf8eaa7eb,3a1fe01360ff8100e7d006b83b77a3e4c01d928c,171.36.92.*,239,20fc675468712705dbf5d3eda94126da,3ddf173a94bd23c326683b6373c75dd4,NaN,mm_10982364_973726_8930541,...,0,0,0,0,1,0,0,0,0,0
2,0,13,b604e3fd054a658ab7ced4285ebf2ef54d2bd890,801d18a056b6fe6b06a794aef17fb0d6daff2414,59.46.106.*,41,trqRTJn7O95I1mKYUV,625d1b5916ea925332c7b326c0574cfa,NaN,1720123646,...,0,0,0,0,0,0,0,0,1,0
3,0,23,0348beeae93e561584c3b50fc9e7746a33048ad7,0d6eaf2259699990e38a1fc5116f112070b9ecdc,114.250.226.*,1,5F97t5E0BTK7XhNrUMpENpn,dedc488b98ca20707bc9a723957e7d1f,NaN,mm_10027070_118039_10308280,...,0,1,0,0,0,0,0,0,0,0
4,0,6,268149c1789bce2bc9798ffd97ec431219bafeb3,a239d9bb642460d974ba67f85e63b8d3e214da0e,183.63.192.*,233,13625cb070ffb306b425cd803c4b7ab4,4199d1227c511fc776b76594dabff9f8,NaN,1120200150,...,0,0,1,0,0,0,0,0,0,0


In [5]:
# Downsample majority class and combine minority class with downsampled majority class
df_majority = pd.concat([resample(train_df[train_df.click==0], replace=False, n_samples=10000, random_state=123), train_df[train_df.click==1]])
 
# Upsample minority class and Combine majority class with upsampled minority class
train_df2 = pd.concat([df_majority, resample(train_df[train_df.click==1], replace=True, n_samples=10000-1793, random_state=123)])
 
# Display new class counts
train_df2.head()

,click,hour,bidid,userid,IP,city,domain,url,urlid,slotid,...,slotvisibility_ThirdView,advertiser_1458,advertiser_2259,advertiser_2261,advertiser_2821,advertiser_2997,advertiser_3358,advertiser_3386,advertiser_3427,advertiser_3476
1148805,0,13,5d00ac4203cfb7176a2dd74609e6df1373d1396d,61a591cdb6e77cb56ec6487b10c05d56f6359362,61.133.143.*,107,b57fdfdc11993baed9c4b4ef265f4717,626bba387bc56e4fc51299dccd5ea9af,NaN,9223372032561121668,...,0,0,0,0,1,0,0,0,0,0
1867729,0,1,eb546d7bd897a45c82acf057c60d7fb1eb55330f,fb3e30c5118829a0b7702957263d61aa1a430216,58.61.14.*,219,trqRTuToMTNUjM9r5rMi,8c96434bb6f4a2aa274a76ad60343f9e,NaN,mm_17967481_3274913_10742094,...,0,0,0,0,0,0,0,0,1,0
354090,0,23,89f91d62f59a8ab688a1f6f0f2df9919fb94c53f,fb9ca42ee00d19355ef5335e0eda46e12c5779e5,110.208.113.*,216,51bd24c0cdd8d076b44125eb461eb457,c4886f515b964c782b16af6ff8fb298d,NaN,9223372032560881844,...,0,0,0,0,1,0,0,0,0,0
642202,0,22,0aa4c41698d3274ed47e2a879eddaab6a5ac5ead,eca38f2004d3205c46ab5e99e1bff1abd303c5e2,124.115.221.*,334,trqRTuMmjocs1m58uG,100ce6f7b506b28c6641a5ca09fe4de5,NaN,mm_10122967_998742_10099299,...,0,1,0,0,0,0,0,0,0,0
1678178,0,12,c9880874d1b486a8e203cab822e4895b3c800a4a,f67980a9c7eedced82160bfe6bbacd52834916d1,1.85.38.*,334,ersbQv1RdoTy1m58uG,ee650ade559fea564c94d1b8b0e0720c,NaN,Sports_F_Width1,...,0,0,0,0,0,0,0,0,1,0


In [6]:
# df1['usertag'] = df1['usertag'].fillna(0)

In [7]:
train_df2.to_csv('train_ver_1.csv')

In [8]:
# s = pd.read_csv('/content/gdrive/My Drive/we_data/train_ver_1.csv')

In [9]:
val_df = pd.read_csv('/Users/tuxiaochi/Desktop/jupyternb/we_data/validation.csv')
val_df.head()

,click,weekday,hour,bidid,userid,useragent,IP,region,city,adexchange,...,slotheight,slotvisibility,slotformat,slotprice,creative,bidprice,payprice,keypage,advertiser,usertag
0,0,4,20,bbcb813b6166538503d8b33a5602d7d72f6019dc,663169f66491c98c69f1f94a5c48fa34aa9fe06f,windows_ie,211.144.203.*,79,79,1.0,...,600,2,1,0,cb7c76e7784031272e37af8e7e9b062c,300,23,bebefa5efe83beee17a3d245e7c5085b,1458,"13866,10111"
1,0,1,21,5a07316c49477cb5d9b4d5aa39c27d6c3be7f92d,a23f0cfab6592c137f796e68fa752fceba08bb1b,windows_chrome,58.247.250.*,79,79,1.0,...,90,0,1,0,c938195f9e404b4f38c7e71bf50263e5,238,75,a3becb967c015e9acb5e72f22b100ef8,3476,"10075,10057,10024,10052,10083,10063,10006,10110"
2,0,4,8,f6ece71dae81d6b16bfb24ad6dd5611472d4c673,5105a013ea54a9706146033e6a138d5234a3803e,windows_ie,117.12.111.*,2,2,2.0,...,250,2,0,5,911b2d84826786018761e8c0b0a3a60c,238,65,d29e59bf0f7f8243858b8183f14d4412,3358,"13866,10024,10059,10063,13776,10083,10006,10111"
3,0,5,15,b4d5c57c9b38ff5a12954fa01e11931b4e6bfbbb,577e9a768ce17e73c61ba7022db927f9ee761830,windows_chrome,61.187.224.*,201,205,2.0,...,280,2,0,5,dc0998c10f8f0b623b5d949e8272e4c7,238,6,d29e59bf0f7f8243858b8183f14d4412,3358,"13866,10057,10006,10063,10110"
4,0,1,18,0899bf144249458ea9c89188473694bf44c7ca15,e4bf79c562745d671b19f2edf7fda89c2e25987f,windows_chrome,117.41.145.*,134,135,2.0,...,200,1,0,5,23485fcd23122d755d38f8c89d46ca56,249,5,7e0e909ea89f707bc20f9585c9072081,3476,"10006,10063,13776,10110"


In [10]:
cat_vars=['region','weekday','useragent','adexchange','slotwidth','slotheight','slotvisibility','advertiser']

for i in cat_vars:
    one_hot = pd.get_dummies(val_df[i],prefix=i)
    val_df = val_df.drop(i,axis = 1)
    val_df = val_df.join(one_hot)

val_df.head()

,click,hour,bidid,userid,IP,city,domain,url,urlid,slotid,...,slotvisibility_ThirdView,advertiser_1458,advertiser_2259,advertiser_2261,advertiser_2821,advertiser_2997,advertiser_3358,advertiser_3386,advertiser_3427,advertiser_3476
0,0,20,bbcb813b6166538503d8b33a5602d7d72f6019dc,663169f66491c98c69f1f94a5c48fa34aa9fe06f,211.144.203.*,79,trqRTvFRLpscFU,5ede68cc7542a34777fb8a1574856b6f,NaN,mm_10027070_2459574_9659312,...,0,1,0,0,0,0,0,0,0,0
1,0,21,5a07316c49477cb5d9b4d5aa39c27d6c3be7f92d,a23f0cfab6592c137f796e68fa752fceba08bb1b,58.247.250.*,79,trqRTu1YP5scFsf,cff326604db401e7d4f895fd747f7e91,NaN,mm_10075660_3500949_11453278,...,0,0,0,0,0,0,0,0,0,1
2,0,8,f6ece71dae81d6b16bfb24ad6dd5611472d4c673,5105a013ea54a9706146033e6a138d5234a3803e,117.12.111.*,2,trqRTv14MqTR1m58uG,93f242f025dc7d424c4ba7002b177df7,NaN,2996225656,...,0,0,0,0,0,0,1,0,0,0
3,0,15,b4d5c57c9b38ff5a12954fa01e11931b4e6bfbbb,577e9a768ce17e73c61ba7022db927f9ee761830,61.187.224.*,205,trqRTvdJMoFoFoc4JKn,40c388f45dbc0e1fc53a8c6c81305de5,NaN,3481684581,...,0,0,0,0,0,0,1,0,0,0
4,0,18,0899bf144249458ea9c89188473694bf44c7ca15,e4bf79c562745d671b19f2edf7fda89c2e25987f,117.41.145.*,135,tKFJeun0BouoXOl4JKTI,fcf53e0e5f09aa8d7e1c9d986cea321,NaN,1120607322,...,0,0,0,0,0,0,0,0,0,1


In [11]:
for i in range(len(train_df.columns.tolist())):
    if train_df.columns.tolist()[i] not in val_df.columns.tolist():
        print('Variable ' + train_df.columns.tolist()[i] + ' to be added to validation set')
        val_df[train_df.columns.tolist()[i]]='0'

Variable useragent_android_ie to be added to validation set
Variable useragent_android_maxthon to be added to validation set
Variable useragent_other_firefox to be added to validation set


In [12]:
val_df.head()

,click,hour,bidid,userid,IP,city,domain,url,urlid,slotid,...,slotvisibility_ThirdView,advertiser_1458,advertiser_2259,advertiser_2261,advertiser_2821,advertiser_2997,advertiser_3358,advertiser_3386,advertiser_3427,advertiser_3476
0,0,20,bbcb813b6166538503d8b33a5602d7d72f6019dc,663169f66491c98c69f1f94a5c48fa34aa9fe06f,211.144.203.*,79,trqRTvFRLpscFU,5ede68cc7542a34777fb8a1574856b6f,NaN,mm_10027070_2459574_9659312,...,0,1,0,0,0,0,0,0,0,0
1,0,21,5a07316c49477cb5d9b4d5aa39c27d6c3be7f92d,a23f0cfab6592c137f796e68fa752fceba08bb1b,58.247.250.*,79,trqRTu1YP5scFsf,cff326604db401e7d4f895fd747f7e91,NaN,mm_10075660_3500949_11453278,...,0,0,0,0,0,0,0,0,0,1
2,0,8,f6ece71dae81d6b16bfb24ad6dd5611472d4c673,5105a013ea54a9706146033e6a138d5234a3803e,117.12.111.*,2,trqRTv14MqTR1m58uG,93f242f025dc7d424c4ba7002b177df7,NaN,2996225656,...,0,0,0,0,0,0,1,0,0,0
3,0,15,b4d5c57c9b38ff5a12954fa01e11931b4e6bfbbb,577e9a768ce17e73c61ba7022db927f9ee761830,61.187.224.*,205,trqRTvdJMoFoFoc4JKn,40c388f45dbc0e1fc53a8c6c81305de5,NaN,3481684581,...,0,0,0,0,0,0,1,0,0,0
4,0,18,0899bf144249458ea9c89188473694bf44c7ca15,e4bf79c562745d671b19f2edf7fda89c2e25987f,117.41.145.*,135,tKFJeun0BouoXOl4JKTI,fcf53e0e5f09aa8d7e1c9d986cea321,NaN,1120607322,...,0,0,0,0,0,0,0,0,0,1


In [13]:
print(len(val_df[val_df['click']==1]), len(val_df[val_df['click']==0]))

202 303723


In [14]:
# Downsample majority class and combine minority class with downsampled majority class
df_majority = pd.concat([resample(val_df[val_df.click==0], replace=False, n_samples=10000, random_state=123), val_df[val_df.click==1]])
 
# Upsample minority class and Combine majority class with upsampled minority class
val_df2 = pd.concat([df_majority, resample(val_df[val_df.click==1], replace=True, n_samples=10000-202, random_state=123)])
 
# Display new class counts
val_df2.head()

,click,hour,bidid,userid,IP,city,domain,url,urlid,slotid,...,slotvisibility_ThirdView,advertiser_1458,advertiser_2259,advertiser_2261,advertiser_2821,advertiser_2997,advertiser_3358,advertiser_3386,advertiser_3427,advertiser_3476
118986,0,0,a352346f8f50a4749d4f460db19893c0c40aece6,896dad661ea92c1e2ba106a6c029797623ef11a3,119.181.181.*,149,trqRTuN-XIuc1mKYUV,b9b27f86cd5409b505b66e27470765af,NaN,724981498,...,0,0,0,0,0,0,0,0,1,0
186165,0,11,f44a0de3bcfd31e8297cf3452544fcf057c55643,a9882283ea026b9cae1b7d465d6fe91fc399dece,222.129.21.*,1,ersbQv1RdoTy1m58uG,772deb2e193778a04e8d246a2042e5ed,NaN,ALLINONE_F_Width1,...,0,0,0,0,0,0,0,0,1,0
115217,0,19,5285ef3f3c209d941df5475bedaaf5532ec3c3e7,9df394089205fb64251303f2e16eadc47302bd13,113.104.13.*,233,NaN,NaN,NaN,1,...,0,0,0,0,0,1,0,0,0,0
214338,0,18,ff572aba54b0b4677ebc154a2a124d61a36bbb53,94810f470ef521a70286ee3cee8b40c08958f6cb,116.9.163.*,245,3FF-e59aG5syJqKbuKz,6bedb3b8e7846afade5237263c21ddb5,NaN,ALLINONE_F_Width1,...,0,0,0,0,0,0,0,1,0,0
176088,0,17,d46609103d4f57176e1270a84d2844816c18862a,3f7baedc1fd871ac4680657d44f333fa849d8592,183.11.103.*,219,eSMJqvNNGqf7kZjKtdpEN0,fbf439c03f54861b5779f78be03e250d,NaN,mm_14481330_2061317_9133530,...,0,1,0,0,0,0,0,0,0,0


In [15]:
print(len(val_df2[val_df2['click']==1]), len(val_df2[val_df2['click']==0]))

10000 10000


In [16]:
train_df.usertag = train_df.usertag.fillna(0)
train_df.head()

,click,hour,bidid,userid,IP,city,domain,url,urlid,slotid,...,slotvisibility_ThirdView,advertiser_1458,advertiser_2259,advertiser_2261,advertiser_2821,advertiser_2997,advertiser_3358,advertiser_3386,advertiser_3427,advertiser_3476
0,0,22,b7bea80521fdecd95d2d761a38c91c3f09618066,2e880fb7d690cf7377b2e42e701728e3f3c0e4c1,125.37.175.*,2,trqRTvKaXTKfgg24JKTI,6447a7dfa30fe6eb410c91860e7c9b45,NaN,2015392487,...,0,0,0,0,0,0,0,0,1,0
1,0,20,4f51205475678f5a124bc76b2c54163bf8eaa7eb,3a1fe01360ff8100e7d006b83b77a3e4c01d928c,171.36.92.*,239,20fc675468712705dbf5d3eda94126da,3ddf173a94bd23c326683b6373c75dd4,NaN,mm_10982364_973726_8930541,...,0,0,0,0,1,0,0,0,0,0
2,0,13,b604e3fd054a658ab7ced4285ebf2ef54d2bd890,801d18a056b6fe6b06a794aef17fb0d6daff2414,59.46.106.*,41,trqRTJn7O95I1mKYUV,625d1b5916ea925332c7b326c0574cfa,NaN,1720123646,...,0,0,0,0,0,0,0,0,1,0
3,0,23,0348beeae93e561584c3b50fc9e7746a33048ad7,0d6eaf2259699990e38a1fc5116f112070b9ecdc,114.250.226.*,1,5F97t5E0BTK7XhNrUMpENpn,dedc488b98ca20707bc9a723957e7d1f,NaN,mm_10027070_118039_10308280,...,0,1,0,0,0,0,0,0,0,0
4,0,6,268149c1789bce2bc9798ffd97ec431219bafeb3,a239d9bb642460d974ba67f85e63b8d3e214da0e,183.63.192.*,233,13625cb070ffb306b425cd803c4b7ab4,4199d1227c511fc776b76594dabff9f8,NaN,1120200150,...,0,0,1,0,0,0,0,0,0,0


In [18]:
count_usertag = []
for i in range(len(train_df)):
    if train_df.usertag[i] == 0:
        count_usertag.append(0)
    else:
        count_usertag.append(len(train_df.usertag[i].split(',')))
count_usertag

[0,
 0,
 4,
 3,
 0,
 5,
 8,
 6,
 4,
 8,
 1,
 8,
 0,
 18,
 1,
 0,
 0,
 4,
 6,
 6,
 6,
 5,
 0,
 8,
 2,
 1,
 7,
 5,
 2,
 10,
 3,
 0,
 2,
 3,
 3,
 0,
 0,
 1,
 8,
 18,
 2,
 12,
 4,
 3,
 3,
 2,
 7,
 4,
 7,
 3,
 16,
 9,
 4,
 2,
 3,
 0,
 0,
 5,
 6,
 5,
 6,
 6,
 0,
 1,
 3,
 8,
 0,
 11,
 3,
 0,
 2,
 6,
 8,
 0,
 13,
 25,
 13,
 0,
 4,
 0,
 3,
 5,
 5,
 2,
 4,
 0,
 2,
 2,
 6,
 2,
 15,
 6,
 0,
 0,
 5,
 1,
 6,
 9,
 3,
 0,
 0,
 17,
 0,
 0,
 1,
 0,
 0,
 7,
 1,
 5,
 7,
 0,
 7,
 5,
 0,
 5,
 8,
 11,
 5,
 7,
 4,
 1,
 0,
 4,
 2,
 0,
 6,
 3,
 0,
 4,
 5,
 6,
 4,
 9,
 0,
 1,
 0,
 9,
 9,
 0,
 6,
 4,
 2,
 3,
 8,
 3,
 6,
 0,
 1,
 11,
 0,
 0,
 0,
 11,
 0,
 4,
 5,
 1,
 3,
 0,
 1,
 0,
 3,
 2,
 4,
 3,
 10,
 5,
 3,
 6,
 9,
 1,
 10,
 0,
 8,
 10,
 0,
 3,
 11,
 5,
 4,
 1,
 10,
 0,
 0,
 4,
 13,
 9,
 10,
 10,
 2,
 2,
 3,
 6,
 7,
 0,
 0,
 0,
 0,
 2,
 6,
 12,
 11,
 12,
 5,
 2,
 1,
 15,
 7,
 9,
 12,
 0,
 21,
 1,
 13,
 7,
 0,
 11,
 9,
 2,
 5,
 1,
 17,
 8,
 0,
 0,
 9,
 4,
 6,
 2,
 5,
 12,
 7,
 5,
 1,
 4,
 2,
 12,
 0,
 5,
 2,
 1,

In [19]:
len(count_usertag) == len(train_df)

True

In [20]:
# without downsampling the validation dataset
# val_df.to_csv('validation_ver_1.csv')

In [21]:
# downsampled validation dataset
# val_df2.to_csv('validation_ver_2.csv')